<a href="https://colab.research.google.com/github/suleiman-odeh/NLP_Project_Team16/blob/main/Qwen2.5_7B/zero_shot_indirect_Qwen2_5_7B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>